# Classificador de tags (I)

In [6]:
import re
import time
import pickle
import urllib2
import numpy as np
from urllib2 import URLError
from bs4 import BeautifulSoup
import selenium.webdriver as webdriver
from selenium.common.exceptions import NoSuchElementException

In [7]:
# load and save lists and dictionaries with pickle
def save_obj(obj, name ):
    with open(''+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [8]:
def restaurant_insta_scraping_location(url):
    browser.get(url)
    try:
        browser.find_element_by_class_name('_oidfu').click()
        more_than_21 = 1
    except NoSuchElementException:
        more_than_21 = 0
    if more_than_21 == 1:
        for i in range(20):
            time.sleep(1.5)
            browser.execute_script("window.scrollTo(0,document.body.scrollHeight);") #Scroll down 
        time.sleep(1)
        html=browser.page_source
        try:
            pictable = browser.find_element_by_xpath('/html/body/span/section/main/article/div[2]/div[1]')
            is_there_pictable = 1
        except NoSuchElementException:
            is_there_pictable = 0
        if is_there_pictable == 1:
            rest_info = []
            for element in pictable.find_elements_by_css_selector("a[class='_8mlbc _vbtk2 _t5r8b']"):
                img = element.find_element_by_class_name('_icyx7')
                text = img.get_attribute('alt')
                tags = [t.replace("#","") for t in re.findall(r'#\w+', text)]
                #dic['url'] = img.get_attribute('src').encode('ascii')
                rest_info.append(tags)
        else:
            rest_info = 0
            
    else: 
        html=browser.page_source
        try:
            pictable = browser.find_element_by_xpath('/html/body/span/section/main/article/div[2]/div[1]')
            is_there_pictable = 1
        except:
            is_there_pictable = 0
        if is_there_pictable == 1:
            rest_info = []
            for element in pictable.find_elements_by_css_selector("a[class='_8mlbc _vbtk2 _t5r8b']"):
                img = element.find_element_by_class_name('_icyx7')
                text = img.get_attribute('alt')
                tags = [t.replace("#","") for t in re.findall(r'#\w+', text)]
                rest_info.append(tags)
        else:
            rest_info = 0
    return rest_info

def listHashtags(string): 
    hashtaglist = []
    while string.find('#',0) != -1:
        m = re.search('(?<=#)\w+', string)
        string = string.replace('#'+m.group(0),'')
        hashtaglist.append(m.group(0))
    hashtaglist = [item.lower() for item in hashtaglist]
    return hashtaglist

def image_insta_scraping(url):
    try:
        source = urllib2.urlopen(url)
        htmlcode = source.read()
        start = htmlcode.find('"caption":') + 12
        end = htmlcode.find('",',start)
        hash_list = listHashtags(htmlcode[start:end])
    except URLError:
        hash_list = 0
    return hash_list

def get_freq_sample(tags,dic):
    N = len(dic)
    freq_sample = [0]*N
    for tag in tags:
        try:
            freq_sample[dic.index(str(tag.lower()))] = 1
        except ValueError:
            None
    return freq_sample

In [9]:
def tag_classifier(url,clf,dic,rest_types):
    if 'locations' in url:
        tags_per_image = restaurant_insta_scraping_location(url)
        if tags_per_image == 0:
            return 'ERROR: non-valid URL'
        freq_samples_per_image = [get_freq_sample(tags,dic) for tags in tags_per_image]
        old_freq_samples_per_image = freq_samples_per_image
        freq_samples_per_image = [item for item in old_freq_samples_per_image if sum(item) != 0]
        if len(freq_samples_per_image) == 0:
            return 'ERROR: I cannot recommend this restaurant because I do not understand the tags of its images.'
        pred_probs_per_image = [clf.predict_proba([freq_samp])[0] for freq_samp in freq_samples_per_image]
        pred_probs_mean = (sum([np.array(pred_probs) for pred_probs in pred_probs_per_image])/len(pred_probs_per_image)).tolist()
        return rest_types[pred_probs_mean.index(max(pred_probs_mean))]
    else:
        if 'instagram.com' in url:
            image_tags = image_insta_scraping(url)
            if image_tags == 0:
                return 'ERROR: non-valid URL'
            freq_samp = get_freq_sample(image_tags,dic)
            if sum(freq_samp) == 0:
                return 'ERROR: I cannot recommend this image because I do not understand its tags.'
            else:
                pred_rest_type = clf.predict([freq_samp])
                pred = clf.predict([freq_samp])[0]
                return rest_types[pred]
        else:
            print 'ERROR: non-valid URL'

In [10]:
# we have, in this folder, classifiers trained with max max_n_of_images images per class.
# for the moment max_n_of_images \in {200,500,1000,2000,9000}
max_n_of_images = 1000

In [11]:
rest_types = load_obj('rest_types')
clf = load_obj('clf_' + str(max_n_of_images))
dic = load_obj('dic_' + str(max_n_of_images))

In [ ]:
# the classifier accepts either instagram image urls or instagram location urls:
#url = 'https://www.instagram.com/explore/locations/238741098/' # LA TAGLIATELLA (ITALIANO)
#url = 'https://www.instagram.com/explore/locations/584346/' #YOSHINO (SUSHI)
#url = 'https://www.instagram.com/explore/locations/375802654/' #POCAS FOTOS
#url = 'https://www.instagram.com/p/BHqpEI7DfaG/' # KFC (FASTFOOD)
#url = 'https://www.instagram.com/explore/locations/88101181/' # VEGGIE
#url = 'https://www.instagram.com/p/BHkzkdmBO2d/' # VEGGIE PIC
#url = 'https://www.instagram.com/p/BGkUS2JIMDg/' #VEGGIE PIC 2
#url = 'https://www.instagram.com/p/BFsNyhQJgqq/' # DA GRECO (ITALIANO)
#url = 'https://www.instagram.com/p/BHwqR2PhIlC/' # MAS PASTA
#url = 'https://www.instagram.com/p/BCP_5NUKwPu/' # CARNE (JALEO)
url = 'https://www.instagram.com/p/BHxoYqdBK6o/' #SUSHI
#url = 'https://www.instagram.com/p/BHxoXXvj7J2/' #SUSHI

path_to_chromedriver = '/Users/AlbertIribarne/Dropbox/UB/bigdata/instagram/chromedriver'
browser = webdriver.Chrome(path_to_chromedriver)

classification = tag_classifier(url,clf,dic,rest_types)
print 'The image / restaurant may correspond to: ' + classification

browser.quit()